In [2]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))

# Add the parent directory to sys.path
sys.path.append(parent_dir)

In [ ]:
!pip install "numpy<2.0.0" pandas matplotlib tqdm torch diffusers scipy zarr

In [3]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import pickle
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
from diffusers.training_utils import EMAModel
from diffusers.optimization import get_scheduler
from tqdm.auto import tqdm
import submodules.data_filter as _df
import diffusion_pipline.data_processing as dproc
import diffusion_pipline.model as md
import submodules.cleaned_file_parser as cfp
import collections


In [ ]:
checkpoint_path = '/home/cam/Documents/raj/diffusion_policy_cam/no-sync/checkpoints/checkpoint_3Body_9_markers_960_SSSS_epoch_119.pth'

checkpoint = torch.load(checkpoint_path)

In [ ]:
checkpoint.keys()

In [ ]:
def predicted_trajectory(first_obs, com_obs_part, statistics, obs_horizon, obs_dim, 
                        pred_horizon, action_horizon, action_dim, 
                        noise_scheduler, num_diffusion_iters,
                        ema_noise_pred_net, device):   
    
    stats = statistics
    # keep a queue of last 2 steps of observations
    obs_deque = collections.deque(
        [first_obs] * obs_horizon, maxlen=obs_horizon)

    # save visualization and rewards
    traj = []
    B = 1
    # stack the last obs_horizon (2) number of observations
    obs_seq = np.stack(obs_deque)
    # normalize observation
    nobs = dproc.normalize_data(obs_seq, stats=stats['obs'])
    # device transfer
    nobs = torch.from_numpy(nobs).to(device, dtype=torch.float32)
    # infer action
    with torch.no_grad():
        # reshape observation to (B,obs_horizon*obs_dim)
        obs_cond = nobs.unsqueeze(0).flatten(start_dim=1)
        
        # initialize action from Guassian noise
        noisy_action = torch.randn(
            (B, pred_horizon, action_dim), device=device)
        naction = noisy_action

        # init scheduler
        noise_scheduler.set_timesteps(num_diffusion_iters)

        for k in noise_scheduler.timesteps:
            # predict noise
            noise_pred = ema_noise_pred_net(
                sample=naction,
                timestep=k,
                global_cond=obs_cond
            )

            # inverse diffusion step (remove noise)
            naction = noise_scheduler.step(
                model_output=noise_pred,
                timestep=k,
                sample=naction
            ).prev_sample

    # unnormalize action
    naction = naction.detach().to('cpu').numpy()
    naction = naction[0]
    action_pred = dproc.unnormalize_data(naction, stats=stats['action'])

    # only take action_horizon number of actions
    start = obs_horizon - 1
    end = start + action_horizon
    action = action_pred[start:end,:]

    for i in range(len(action)):
        action_last = list(action[i])
        obs_deque.append(action_last + com_obs_part)
        traj.append(action_last + com_obs_part)


    return traj

In [ ]:
# Parameters corrsponding to
num_epochs =checkpoint['num_epochs']
obs_dim = checkpoint['obs_dim']
action_dim = checkpoint['action_dim']
# parameters
pred_horizon = checkpoint['pred_horizon']
obs_horizon = checkpoint['obs_horizon']
action_horizon = checkpoint['action_horizon']
target_fps = checkpoint['target_fps']

action_item = checkpoint['action_item']
obs_item = checkpoint['obs_item']


# create network object
noise_pred_net = md.ConditionalUnet1D(
    input_dim=action_dim,
    global_cond_dim=obs_dim*obs_horizon
)

# example inputs
noised_action = torch.randn((1, pred_horizon, action_dim))
obs = torch.zeros((1, obs_horizon, obs_dim))
diffusion_iter = torch.zeros((1,))

# the noise prediction network
# takes noisy action, diffusion iteration and observation as input
# predicts the noise added to action
noise = noise_pred_net(
    sample=noised_action,
    timestep=diffusion_iter,
    global_cond=obs.flatten(start_dim=1))

# illustration of removing noise
# the actual noise removal is performed by NoiseScheduler
# and is dependent on the diffusion noise schedule
denoised_action = noised_action - noise

# for this demo, we use DDPMScheduler with 100 diffusion iterations
num_diffusion_iters = 100
noise_scheduler = DDPMScheduler(
    num_train_timesteps=num_diffusion_iters,
    # the choise of beta schedule has big impact on performance
    # we found squared cosine works the best
    beta_schedule='squaredcos_cap_v2',
    # clip output to [-1,1] to improve stability
    clip_sample=True,
    # our network predicts noise (instead of denoised action)
    prediction_type='epsilon'
)

# device transfer
device = torch.device('cuda')
_ = noise_pred_net.to(device)

# Exponential Moving Average
# accelerates training and improves stability
# holds a copy of the model weights
ema = EMAModel(
    parameters=noise_pred_net.parameters(),
    power=0.75)

# Standard ADAM optimizer
# Note that EMA parametesr are not optimized
optimizer = torch.optim.AdamW(
    params=noise_pred_net.parameters(),
    lr=1e-4, weight_decay=1e-6)

# Cosine LR schedule with linear warmup
lr_scheduler = get_scheduler(
    name='cosine',
    optimizer=optimizer,
    num_warmup_steps=200,
    num_training_steps=checkpoint['len_dataloader'] * num_epochs
)

ema_noise_pred_net = noise_pred_net

In [ ]:
noise_pred_net.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
lr_scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
ema.load_state_dict(checkpoint['ema_state_dict'])
start_epoch = checkpoint['epoch'] + 1

In [5]:

    

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Use", torch.cuda.device_count(), 'gpus')

Use 1 gpus
